# Influence Diagnostics in Meta-Analysis

This notebook demonstrates advanced influence diagnostics and outlier detection methods in PyMeta.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pymeta as pm
from scipy import stats

plt.style.use('seaborn-v0_8')
%matplotlib inline

## Generate Data with Outliers

In [ ]:
# Create sample data with some outlying studies
np.random.seed(123)

n_studies = 20
studies = [f'Study_{i+1:02d}' for i in range(n_studies)]

# Generate effect sizes - most studies around 0.3, but some outliers
effect_sizes = np.random.normal(0.3, 0.1, n_studies)

# Add some outliers
outlier_indices = [5, 12, 17]
effect_sizes[outlier_indices] = [0.8, -0.2, 1.1]  # Clear outliers

# Generate variances (inversely related to sample size)
sample_sizes = np.random.randint(50, 300, n_studies)
variances = (2.0 / np.sqrt(sample_sizes)) * np.random.uniform(0.8, 1.2, n_studies)

# One study with very large variance (poor quality)
variances[8] = 0.15

data = pd.DataFrame({
    'study': studies,
    'effect_size': effect_sizes,
    'variance': variances,
    'standard_error': np.sqrt(variances),
    'sample_size': sample_sizes,
    'year': np.random.randint(2010, 2024, n_studies),
    'quality': np.random.choice(['High', 'Medium', 'Low'], n_studies)
})

print(f"Created dataset with {len(data)} studies")
print(f"Outlier studies: {[studies[i] for i in outlier_indices]}")
data.head()

## Basic Meta-Analysis

In [ ]:
# Perform initial meta-analysis
result = pm.meta_analysis(data, model='random', tau2_method='reml')

print("Initial Meta-Analysis Results:")
print("-" * 35)
print(f"Overall effect: {result.overall_effect:.3f}")
print(f"95% CI: [{result.ci_lower:.3f}, {result.ci_upper:.3f}]")
print(f"Tau²: {result.tau_squared:.3f}")
print(f"I²: {result.i_squared:.1f}%")
print(f"Q-statistic: {result.q_statistic:.2f} (p = {result.q_pvalue:.3f})")

In [ ]:
# Initial forest plot
fig, ax = plt.subplots(figsize=(12, 10))
pm.forest_plot(result, ax=ax, title='Forest Plot - All Studies')
plt.tight_layout()
plt.show()

## 1. Leave-One-Out Analysis

In [ ]:
# Leave-one-out influence analysis
loo_result = pm.leave_one_out(data)

print("Leave-One-Out Analysis:")
print("-" * 25)
print(f"Effect size range: {loo_result.min_effect:.3f} to {loo_result.max_effect:.3f}")
print(f"Standard error range: {loo_result.min_se:.3f} to {loo_result.max_se:.3f}")
print(f"Most influential study: {loo_result.most_influential_study}")
print(f"Maximum influence: {loo_result.max_influence:.3f}")

# Display top 5 most influential studies
print("\nTop 5 Most Influential Studies:")
influence_df = loo_result.influence_data.sort_values('influence', ascending=False).head()
for _, row in influence_df.iterrows():
    print(f"  {row['study']}: Influence = {row['influence']:.3f}")

In [ ]:
# Visualize leave-one-out results
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Influence plot
pm.influence_plot(loo_result, ax=axes[0,0], title='Leave-One-Out Influence')

# Effect size change plot
pm.loo_effect_plot(loo_result, ax=axes[0,1], title='Effect Size Changes')

# Standard error change plot
pm.loo_se_plot(loo_result, ax=axes[1,0], title='Standard Error Changes')

# Combined influence metrics
pm.influence_metrics_plot(loo_result, ax=axes[1,1], title='Multiple Influence Metrics')

plt.tight_layout()
plt.show()

## 2. Studentized Residuals

In [ ]:
# Calculate studentized residuals
residuals_result = pm.studentized_residuals(data)

print("Studentized Residuals Analysis:")
print("-" * 32)
print(f"Mean residual: {residuals_result.residuals.mean():.3f}")
print(f"SD of residuals: {residuals_result.residuals.std():.3f}")

# Identify outliers (|residual| > 2 or 3)
moderate_outliers = residuals_result.outliers_moderate  # |residual| > 2
extreme_outliers = residuals_result.outliers_extreme    # |residual| > 3

print(f"\nModerate outliers (|residual| > 2): {len(moderate_outliers)}")
for study in moderate_outliers:
    idx = data[data['study'] == study].index[0]
    residual = residuals_result.residuals.iloc[idx]
    print(f"  {study}: {residual:.3f}")

print(f"\nExtreme outliers (|residual| > 3): {len(extreme_outliers)}")
for study in extreme_outliers:
    idx = data[data['study'] == study].index[0]
    residual = residuals_result.residuals.iloc[idx]
    print(f"  {study}: {residual:.3f}")

In [ ]:
# Visualize residuals
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# Residuals vs fitted values
pm.residual_plot(residuals_result, ax=axes[0,0], title='Residuals vs Fitted')

# Q-Q plot of residuals
pm.qq_plot(residuals_result, ax=axes[0,1], title='Q-Q Plot of Residuals')

# Histogram of residuals
axes[1,0].hist(residuals_result.residuals, bins=10, alpha=0.7, edgecolor='black')
axes[1,0].axvline(0, color='red', linestyle='--', alpha=0.7)
axes[1,0].axvline(2, color='orange', linestyle='--', alpha=0.7, label='Moderate outlier')
axes[1,0].axvline(-2, color='orange', linestyle='--', alpha=0.7)
axes[1,0].axvline(3, color='red', linestyle='--', alpha=0.7, label='Extreme outlier')
axes[1,0].axvline(-3, color='red', linestyle='--', alpha=0.7)
axes[1,0].set_xlabel('Studentized Residual')
axes[1,0].set_ylabel('Frequency')
axes[1,0].set_title('Distribution of Residuals')
axes[1,0].legend()

# Residuals vs precision
pm.residual_precision_plot(residuals_result, ax=axes[1,1], title='Residuals vs Precision')

plt.tight_layout()
plt.show()

## 3. Baujat Analysis

In [ ]:
# Baujat analysis - identifies studies contributing to heterogeneity
baujat_result = pm.baujat_analysis(data)

print("Baujat Analysis:")
print("-" * 16)
print("Studies with high contribution to heterogeneity:")

# Identify studies in upper right quadrant
high_contrib = baujat_result.high_heterogeneity_contribution
for study in high_contrib:
    idx = data[data['study'] == study].index[0]
    contrib = baujat_result.heterogeneity_contribution.iloc[idx]
    influence = baujat_result.overall_influence.iloc[idx]
    print(f"  {study}: H² contribution = {contrib:.3f}, Influence = {influence:.3f}")

In [ ]:
# Baujat plot with study labels
fig, ax = plt.subplots(figsize=(10, 8))
pm.baujat_plot(result, ax=ax, show_labels=True, title='Baujat Plot with Study Labels')
plt.tight_layout()
plt.show()

## 4. Cook's Distance and DFBETAS

In [ ]:
# Calculate Cook's distance and DFBETAS
influence_stats = pm.influence_statistics(data)

print("Influence Statistics:")
print("-" * 20)

# Cook's Distance threshold (typically > 4/n)
cooks_threshold = 4 / len(data)
high_cooks = influence_stats.cooks_distance > cooks_threshold

print(f"Cook's Distance threshold: {cooks_threshold:.3f}")
print(f"Studies with high Cook's Distance:")
for i, study in enumerate(data['study'][high_cooks]):
    cooks_d = influence_stats.cooks_distance.iloc[i]
    print(f"  {study}: {cooks_d:.3f}")

# DFBETAS threshold (typically > 2/sqrt(n))
dfbetas_threshold = 2 / np.sqrt(len(data))
high_dfbetas = np.abs(influence_stats.dfbetas) > dfbetas_threshold

print(f"\nDFBETAS threshold: ±{dfbetas_threshold:.3f}")
print(f"Studies with high DFBETAS:")
for i, study in enumerate(data['study'][high_dfbetas]):
    dfbetas = influence_stats.dfbetas.iloc[i]
    print(f"  {study}: {dfbetas:.3f}")

In [ ]:
# Visualize influence statistics
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# Cook's Distance
pm.cooks_distance_plot(influence_stats, ax=axes[0,0], title="Cook's Distance")

# DFBETAS
pm.dfbetas_plot(influence_stats, ax=axes[0,1], title='DFBETAS')

# Leverage vs Residuals
pm.leverage_residual_plot(influence_stats, ax=axes[1,0], title='Leverage vs Residuals')

# Influence bubble plot
pm.influence_bubble_plot(influence_stats, ax=axes[1,1], title='Influence Bubble Plot')

plt.tight_layout()
plt.show()

## 5. Outlier Detection Summary

In [ ]:
# Comprehensive outlier detection
outlier_summary = pm.outlier_detection_summary(data)

print("Comprehensive Outlier Detection Summary:")
print("=" * 42)

outlier_df = outlier_summary.summary_table
print(outlier_df.to_string())

# Consensus outliers (flagged by multiple methods)
consensus_outliers = outlier_summary.consensus_outliers
print(f"\nConsensus outliers (flagged by ≥3 methods): {len(consensus_outliers)}")
for study in consensus_outliers:
    methods = outlier_df.loc[outlier_df['study'] == study, 'methods_flagged'].iloc[0]
    print(f"  {study}: {methods} methods")

## 6. Sensitivity Analysis: Removing Outliers

In [ ]:
# Compare results with and without outliers
print("Sensitivity Analysis: Impact of Outlier Removal")
print("=" * 48)

# Original analysis
original_result = pm.meta_analysis(data, model='random')
print(f"Original analysis (n={len(data)}):")
print(f"  Effect: {original_result.overall_effect:.3f} [{original_result.ci_lower:.3f}, {original_result.ci_upper:.3f}]")
print(f"  Tau²: {original_result.tau_squared:.3f}, I²: {original_result.i_squared:.1f}%")

# Remove consensus outliers
if len(consensus_outliers) > 0:
    data_no_outliers = data[~data['study'].isin(consensus_outliers)].copy()
    
    outlier_removed_result = pm.meta_analysis(data_no_outliers, model='random')
    print(f"\nOutliers removed (n={len(data_no_outliers)}):")
    print(f"  Effect: {outlier_removed_result.overall_effect:.3f} [{outlier_removed_result.ci_lower:.3f}, {outlier_removed_result.ci_upper:.3f}]")
    print(f"  Tau²: {outlier_removed_result.tau_squared:.3f}, I²: {outlier_removed_result.i_squared:.1f}%")
    
    # Calculate impact
    effect_change = abs(outlier_removed_result.overall_effect - original_result.overall_effect)
    tau2_change = abs(outlier_removed_result.tau_squared - original_result.tau_squared)
    
    print(f"\nImpact of outlier removal:")
    print(f"  Effect size change: {effect_change:.3f}")
    print(f"  Tau² change: {tau2_change:.3f}")
    print(f"  I² change: {outlier_removed_result.i_squared - original_result.i_squared:.1f} percentage points")
else:
    print("\nNo consensus outliers identified.")
    data_no_outliers = data.copy()
    outlier_removed_result = original_result

In [ ]:
# Side-by-side forest plots
fig, axes = plt.subplots(1, 2, figsize=(20, 10))

# Original data
pm.forest_plot(original_result, ax=axes[0], title=f'All Studies (n={len(data)})')

# Without outliers
pm.forest_plot(outlier_removed_result, ax=axes[1], title=f'Outliers Removed (n={len(data_no_outliers)})')

plt.tight_layout()
plt.show()

## 7. Advanced Influence Diagnostics

In [ ]:
# Advanced diagnostics
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Hat values (leverage)
pm.leverage_plot(influence_stats, ax=axes[0,0], title='Leverage (Hat Values)')

# Standardized residuals vs leverage
pm.standard_residual_leverage_plot(influence_stats, ax=axes[0,1], 
                                  title='Standardized Residuals vs Leverage')

# DFFITS
pm.dffits_plot(influence_stats, ax=axes[1,0], title='DFFITS')

# Covariance ratio
pm.covariance_ratio_plot(influence_stats, ax=axes[1,1], title='Covariance Ratio')

plt.tight_layout()
plt.show()

## 8. Influence on Heterogeneity

In [ ]:
# Examine influence on heterogeneity measures
het_influence = pm.heterogeneity_influence(data)

print("Influence on Heterogeneity Measures:")
print("-" * 36)
print(f"Original I²: {original_result.i_squared:.1f}%")
print(f"Original Tau²: {original_result.tau_squared:.3f}")
print(f"Original Q: {original_result.q_statistic:.2f}")
print()

print("Studies with largest impact on heterogeneity:")
top_het_influence = het_influence.sort_values('tau2_change', ascending=False).head(5)
for _, row in top_het_influence.iterrows():
    print(f"  {row['study']}: ΔTau² = {row['tau2_change']:.3f}, ΔI² = {row['i2_change']:.1f}%")

In [ ]:
# Plot heterogeneity influence
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

pm.tau2_influence_plot(het_influence, ax=axes[0], title='Influence on Tau²')
pm.i2_influence_plot(het_influence, ax=axes[1], title='Influence on I²')

plt.tight_layout()
plt.show()

## Conclusions and Recommendations

### Summary of Influence Diagnostics

This notebook demonstrated comprehensive influence diagnostics for meta-analysis:

1. **Leave-One-Out Analysis** - Systematic removal of each study
2. **Studentized Residuals** - Standardized measures of study deviation
3. **Baujat Analysis** - Identification of heterogeneity contributors
4. **Cook's Distance & DFBETAS** - Overall influence measures
5. **Comprehensive Outlier Detection** - Multiple method consensus
6. **Sensitivity Analysis** - Impact assessment of outlier removal
7. **Advanced Diagnostics** - Leverage, DFFITS, covariance ratios
8. **Heterogeneity Influence** - Impact on between-study variance

### Key Findings:
- Multiple outlying studies were detected across different methods
- Consensus outliers significantly impact both effect size and heterogeneity
- Removal of outliers leads to more precise estimates with reduced heterogeneity

### Recommendations:
1. Always conduct influence diagnostics in meta-analysis
2. Use multiple detection methods for robust identification
3. Investigate reasons for outlying results before removal
4. Report sensitivity analyses with and without influential studies
5. Consider subgroup analysis if outliers share characteristics